In [1]:
import urllib.request
import numpy as np
import json
import string
from sklearn import metrics as sk_m
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn as sk
import matplotlib.pyplot as plt
import random as random
from scipy.stats import norm
import statistics as stat

### Load original embedding

In [2]:
# URL to retrive pre-trained 300 dimensional gloVe embedding
embedding_300_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors.txt"

def read_embedding(url, skip_first = False):
    """Function to read out an embedding
    Input: url: url to embedding
    
    Returns: vocab: list of words in the embedding
             w2id: dictionary mapping words to ids
             embedding: array storing the word vectors,
                           row corresponds to word id"""
    # Open url
    data = urllib.request.urlopen(url)
    vocab = []
    embedding = []
    
    # Each line contains one word and its embedding
    for i, line in enumerate(data):
        if skip_first:
            if i == 0:
                continue
        #if len(line) == 301:
        line = line.decode()
        # Split by spaces
        split = line.split()
        # First element(== the word) is added to vocabulary
        vocab.append(split[0])
        # All other elements(embedding vectors) are added to vectors
        embedding.append([float(elem) for elem in split[1:]])
    
    # Create a dictionary with word-id pairs based on the order
    w2id = {w: i for i, w in enumerate(vocab)}
    # Vectors are converted into an array
    embedding = np.array(embedding).astype(float)
    
    return vocab, w2id, embedding
    
vocab_original, w2id_original, embedding_original = read_embedding(embedding_300_url)

In [3]:
def hasDigit(word):
    """Checks if a string contains any digits"""
    return any(char.isdigit() for char in word)

def hasSpecialChar(word):
    """Checks if a string contains special characters(except "_")"""
    special_characters = "!@#$%^&*()-+?=,<>/."
    return any(char in special_characters for char in word)

In [4]:
def restrict_vocab(vocab, w2id, embedding):
    """Limits the vocab by removing words containing digits or special characters
    Input: vocab: list of words in the embedding
           w2id: dictionary mapping words to ids
           embedding: array storing the word vectors
           
    Returns: limit_vocab: list of words in vocab that do not include digits or special characters
             limit_w2id: dictionary mapping words in limit_vocab to new ids
             limit_embedding: array storing the word vectors of the words in limit_vocab only"""
    limit_vocab = []
    limit_embedding = []
    
    for word in vocab:
        # If word includes either a digit or a special character move on to next word
        if hasDigit(word) or hasSpecialChar(word):
            continue
        # Else add word to limit_vocab and its embedding to limit_embedding    
        limit_vocab.append(word)
        limit_embedding.append(embedding[w2id[word]])
        
    # Convert embedding into an array    
    limit_embedding = np.array(limit_embedding).astype(float)
    # Create new dictionary containing only the words in limit_vocab and their new ids
    limit_w2id = {word: i for i, word in enumerate(limit_vocab)}
    
    return limit_vocab, limit_w2id, limit_embedding

In [5]:
vocab, w2id, embedding = restrict_vocab(vocab_original, w2id_original, embedding_original)
print("Original vocab size: ", len(vocab_original))
print("Restricted vocab size: ", len(vocab))

Original vocab size:  322636
Restricted vocab size:  314952


In [6]:
def exclude_vocab(vocab, exclude):
    """Function to exclude specific words from vocabulary
    Input: vocab: list of words in the embedding
           exclude: list of words to exclude from the vocabulary
           
    Returns: limited_vocab: vocab without the words in exclude"""
    # Create copy of vocab
    limited_vocab = vocab.copy()
    # For all words that are in exclude and vocab
    for word in exclude:
        if word in limited_vocab:
            # Remove word from vocab
            limited_vocab.remove(word)
            
    return limited_vocab

In [7]:
# URL to female specific words as listed by the authors
female_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/female_word_file.txt"
female_words_data = urllib.request.urlopen(female_words_url)

# List of female words
female_words = []
for line in female_words_data:
    line = line.decode()
    line = line.split()
    female_words.append(line[0])

In [8]:
# URL to male specific words as listed by the authors
male_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/male_word_file.txt"
male_words_data = urllib.request.urlopen(male_words_url)

# List of male words
male_words = []
for line in male_words_data:
    line = line.decode()
    line = line.split()
    male_words.append(line[0])

In [9]:
# Create List with female - male pairs from female-male specific words
female_male_pairs = []
for i, female in enumerate(female_words):
    female_male_pairs.append([female, male_words[i]])

In [10]:
# URLs to the files storing gender specific words as listed by the authors
gender_specific_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/gender_specific_full.json"

# Empty list to accumulate gender specific words plus additional list after lowercasing
gender_specific_original = []
gender_specific = []


# Read out URL and add further gender specific words
with urllib.request.urlopen(gender_specific_url) as f:
    gender_specific_original.extend(json.load(f))

# Add lower case words to second list
for word in gender_specific_original:
    gender_specific.append(word.lower())

In [11]:
# URL to the file storing definitional pairs as listed by the authors
definitial_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/definitional_pairs.json"

# Empty list to store definitional pairs plus additional list after lowercasing
definitional_pairs_original = []
definitional_pairs = []


# Read out url and add pairs in list
with urllib.request.urlopen(definitial_pairs_url) as f:
    definitional_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in definitional_pairs_original:
    definitional_pairs.append([w1.lower(), w2.lower()])


# Create list of single words instead of pairs  
definitional_words = []
for pair in definitional_pairs:
    for word in pair:
        definitional_words.append(word)
        

In [12]:
# URL to the file storing the equalize pairs as listed by the authors
equalize_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/equalize_pairs.json"

# Empty list to store equalize pairs plus additional list after lowercasing
equalize_pairs_original = []
equalize_pairs = []

# Read out URL and add pairs to list
with urllib.request.urlopen(equalize_pairs_url) as f:
    equalize_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in equalize_pairs_original:
    equalize_pairs.append([w1.lower(), w2.lower()])
    
# Create list of single words instead of pairs
equalize_words = []
for pair in equalize_pairs:
    for word in pair:
        equalize_words.append(word)

In [13]:
# List of all gender specific words included in 
# female words, male words, gender specific words, equalize words and definitional words
exclude_words = list(set(female_words + male_words + gender_specific + definitional_words + equalize_words))

In [14]:
# Remove gender specific words from the embedding to obtain vocabulary of neutral words
vocab_neutral = exclude_vocab(vocab, exclude_words)
print("Vocab size: ", len(vocab))
print("Neutral vocab size: ", len(vocab_neutral))

Vocab size:  314952
Neutral vocab size:  314293


In [15]:
def embed(word, w2id=w2id, embedding=embedding):
    return embedding[w2id[word]]

### Load further embeddings

In [ ]:
embedding_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors300.txt"
vocab_gn_original, w2id_gn_original, embedding_gn_original = read_embedding(embedding_gn_url)
vocab_gn, w2id_gn, embedding_gn = restrict_vocab(vocab_gn_original, w2id_gn_original, embedding_gn_original)

In [ ]:
embedding_hd_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd.txt"
vocab_hd_original, w2id_hd_original, embedding_hd_original = read_embedding(embedding_hd_url)
vocab_hd, w2id_hd, embedding_hd = restrict_vocab(vocab_hd_original, w2id_hd_original, embedding_hd_original)

In [ ]:
embedding_hd_a_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd_a.txt"
vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original = read_embedding(embedding_hd_a_url)
vocab_hd_a, w2id_hd_a, embedding_hd_a = restrict_vocab(vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original)

In [ ]:
embedding_gp_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_glove.txt"
vocab_gp_original, w2id_gp_original, embedding_gp_original = read_embedding(embedding_gp_url, skip_first = True)
vocab_gp, w2id_gp, embedding_gp = restrict_vocab(vocab_gp_original, w2id_gp_original, embedding_gp_original)

In [38]:
embedding_gp_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_gn_glove.txt"
vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original = read_embedding(embedding_gp_gn_url, skip_first = True)
vocab_gp_gn, w2id_gp_gn, embedding_gp_gn = restrict_vocab(vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original)
print("Original vocab size: ", len(vocab_gp_gn_original))
print("Restricted vocab size: ", len(vocab_gp_gn))

Original vocab size:  322636
Restricted vocab size:  314952


### Gender Subspace

In [59]:
def idtfy_gender_subspace(word_sets, w2id, defining_sets, embedding, k=1):
    """
    identifies the bias (gender) subspace following Bolukbasi et al. 2016
    
    takes
    word_sets: vocabulary
    w2id: a dictionary to translate words contained in the vocabulary into their corresponding IDs
    defining_sets: N defining sets (pairs if I=2) consisting of I words that differ mainly on the bias (gender) direction
    embedding: the embedding of the vocabulary
    k: an integer parameters that defines how many rows of SVD(C) constitute the bias (gender) subspace B, bias (gender) direction if k=1
    
    returns
    bias_subspace: linear bias (gender) subspace (direction) that is assumed to capture most of the gender bias (denoted as B in Bolukbasi et al. 2016)
    """
    
    C = []
    for female_word, male_word in defining_sets:
        mean = (embed(female_word) + embed(male_word)) /2
        C.append(embed(female_word) - mean)
        C.append(embed(male_word) - mean)
    C = np.array(C)
    
    # applying PCA is the same as SVD when interpreting C as covariance matrix (Vargas & Cotterell 2020)
    pca = PCA(n_components = 10)
    pca.fit(C)

    # take the first k pcas (first for gender direction)
    B = []
    for i in range(k):
        B.append(pca.components_[i])
    B = np.array(B).flatten()

    
    #print("new_B")
    #array = np.ndarray((10,2,300))
    #i=0
    #array_two = np.zeros((10,300,300))
    #for j, d_pair in enumerate(definitional_pairs):
    #    for i, word in enumerate(d_pair):
    #        # fill array with embeddings
    #        array[j][i] = embedding[w2id[word]]
            #i = i+1
        # print(array[j][0].shape)
        # calculate covariance between embeddings of same definitional pair?
    #    array_two[j]=np.cov(np.transpose(array[j]))
        
    
    return B

In [60]:
gender_subspace = idtfy_gender_subspace(vocab, w2id, definitional_pairs, embedding)

### most biased 500 words

In [61]:
# most biased male and female words
def most_biased(embedding, B, k=500):
    # small x, else memory issues
    x = 50000
    all_biased = np.ndarray((x,1))
    for i, word in enumerate(embedding):
        if i < x:
            all_biased[i] = (sk_m.pairwise.cosine_similarity(word.reshape(1, 300), B.reshape(1, 300)))[0]
            # print(sk_m.pairwise.cosine_similarity(word.reshape(1,300), B)[0])
    #print(all_biased)
    most_biased_f = []
    most_biased_m = []
    for word in range(k):
        # female words
        fb_index = np.argmin(all_biased)
        most_biased_f.append(fb_index)
        all_biased[fb_index] = 0
        # male words
        mb_index = np.argmax(all_biased)
        most_biased_m.append(mb_index)
        all_biased[mb_index] = 0
    #print(most_biased_f, most_biased_m)
    return most_biased_f, most_biased_m

In [62]:
index_f, index_m = most_biased(embedding, gender_subspace)

In [63]:
female_most_biased = [vocab[i] for i in index_f]
print("female", female_most_biased)
male_most_biased = [vocab[i] for i in index_m]
print("male", male_most_biased)

female ['john', 'himself', 'his', 'brother', 'led', 'son', 'colonel', 'successor', 'nephew', 'footballing', 'sir', 'uncle', 'general', 'brothers', 'elway', 'he', 'tackle', 'linemen', 'nhl', 'deere', 'journeyman', 'governorship', 'brigadier', 'apprenticed', 'father', 'hooker', 'punter', 'marshal', 'captaincy', 'daud', 'generals', 'man', 'mayall', 'succeeded', 'henry', 'balliol', 'julius', 'leaguer', 'william', 'appointed', 'godfrey', 'captain', 'wingman', 'successors', 'trombonist', 'predecessor', 'mahdi', 'dibiase', 'fullback', 'domnall', 'papacy', 'inventor', 'engineer', 'military', 'defensive', 'linebacker', 'grandson', 'mcculloch', 'pontificate', 'dawkins', 'legate', 'james', 'mechanic', 'invented', 'ernest', 'surveyor', 'iii', 'muhammed', 'commander', 'czw', 'cardinal', 'irvin', 'receiver', 'sulayman', 'bulls', 'command', 'ibn', 'sayyid', 'dso', 'greats', 'under', 'plow', 'sax', 'robert', 'manager', 'explorer', 'decepticon', 'archbishop', 'grandfather', 'quarterback', 'klitschko', 

### Double Hard Debias

The vector projection of a vector a on (or onto) a nonzero vector b, (also known as the vector component or vector resolution of a in the direction of b), is the orthogonal projection of a onto a straight line parallel to b. It is a vector parallel to b, defined as:

${\displaystyle \mathbf a_{1}} =  a_{1} {\mathbf {\hat {b}} \,} $

where $a_{1}$ is a scalar, called the scalar projection of a onto b, and ${\mathbf {\hat {b}} \,}$ is the unit vector in the direction of b.

In turn, the scalar projection is defined as:[2]

$    a_{1} =  a ⋅ {\mathbf {\hat {b}} \,}= a ⋅ \frac{b}{‖ b ‖} $

where the operator ⋅ denotes a dot product, ‖b‖ is the length of b.

The **vector component** or vector resolute of a perpendicular to b, sometimes also called the **vector rejection of a from b** is the orthogonal projection of a onto the plane (or, in general, hyperplane) orthogonal to b. Both the projection $a_{1}$ and rejection $a_{2}$ of a vector a are vectors, and their sum is equal to a,[1] which implies that the rejection is given by: 

$a_{2} = a − a_{1}$ 

In [169]:
def double_hard_debias(words, index_m, index_f, w2id):
    """Double Hard Debias:
    
    words: word embeddings of some corpus
    index_m: indices of most biased male words 
    index_f: indices of most biased female words 
    w2id:
    """
    # print(words[index_m[0]])
    males = np.asarray([words[i] for i in index_m])
    females = np.asarray([words[i] for i in index_f])    
    
    # decentralize all of the embeddings
    # first calculate mean over full vocab
    mu = ((len(words)**(-1)) * np.sum(words, axis=0)).reshape(1,300)
    
    # restrict corpus for PCA due to Error:
    # Unable to allocate 721. MiB for an array with shape (314952, 300) and data type float64
    #x = 1000
    
    words_decen = np.zeros((len(words),300), dtype='float32') # chose a smaller data type due to memory error
    # then subtract mean from each word embedding
    for index, embedding in enumerate(words):
        #if index < len(words):
        # print(index,":",embedding)
        words_decen[index] = embedding - mu
    
    #print("decentralized:",words_decen.shape)
    #print("origin:",words)
        
    # discover the frequency direction    
    #2. for all decentralized embeddings: compute PCA
    #princ_comp = np.asarray(pca_tft(words_decen))
    #print("Principal Components:",princ_comp)
    pca_freq = PCA().fit(words_decen)
    # print("250",pca_freq.components_[250])
    # print("251",pca_freq.components_[251])
    # print(pca.components_)
    # princ_comp = pca.components_
    #print("Sklearn PCs:", pca_freq.components_.shape)

    evaluations = []

    #3. for all principal components:
    # in implementation of paper only look at 20 first PCs
    for i, pc in enumerate(pca_freq.components_):
        if i < 20:
            male_proj = np.zeros((len(males),300))
            male_debias = np.zeros((len(males),300))
            female_proj = np.zeros((len(females),300))
            female_debias = np.zeros((len(females),300))
    
        
            #pc= pc.reshape(1,300)
        
            for index, male in enumerate(males):
            #male embedding = decentralized embedding - projected embedding into direction of PC
                # print(male.shape)
                #male.reshape((1,300))
                #print(((male @np.transpose(pc.reshape(1,300)))*pc.reshape(1,300)).shape)
                #print((male-mu).shape, ((np.transpose(pc.reshape(1,300))*male)@pc).shape)
                #print("pc", pc.shape, np.transpose(pc.reshape(300,1)).shape)
                #male_proj[index] = (male - mu) - ((male @ np.transpose(unit_vec(pc))))
                male_proj[index] = w_orth(male, pc)
                #with all new male embeddings: HardDebias
                #print(male_proj[index].shape)
                #male_debias[index] = hard_debias(male_proj[index])
                male_debias[index] = w_orth(male_proj[index], gender_subspace)
                #print(male_debias[index].shape)
            
            #print(male_proj == male_debias)
        
            for index, female in enumerate(females):
            #female embedding = decentralized embedding - projected original (?) embedding into direction of PC
                #female.reshape((1,300))
                female_proj[index] = w_orth(female, pc)
                #with all new female embeddings: HardDebias
                #female_debias[index] = hard_debias(female_proj[index])
                female_debias[index] = w_orth(female_proj[index], gender_subspace)
    
            #for all HardDebiased embeddings: KMeansClustering (2)
            #for clustered embeddings: compute gender alignment accuracy
            #4. store evaluations for each principal components
            evaluations.append(align_acc(male_debias, female_debias))
    print("eval:",evaluations)
    
    #5. evaluate which PC lead to most random cluster (evaluation smallest (close to 0.5), used second PC)
    #best_eval = evaluations.index(np.min(evaluations))
    print(np.argmin(evaluations))
    best_pc = pca_freq.components_[np.argmin(evaluations)]
    #print("Best PC:",best_pc,"with evaluation:",evaluations[best_eval])

    first_debias = np.zeros((words.shape))
    #6. for all decentralized embeddings: remove that PC-direction
    for index,word in enumerate(words_decen):
        first_debias[index] = w_orth(word, best_pc)
    
    #7. for all new embeddings: HardDebias
    #double_debias = np.zeros((words.shape))
    #for index,word in enumerate(first_debias):
    #    double_debias[index] = hard_debias(word)
    double_debias = hard_debias(first_debias)

    return double_debias


In [170]:
def unit_vec(vector):
    """calculates unit vector of passed vector"""
    
    unit = np.linalg.norm(vector)
    if unit != 0 and np.isnan(unit) == False :
        return vector/unit
    return vector   

In [171]:
#Gender alignment accuracy/ Neighborhood Metric:
def align_acc(males, females):
    """bias measurement using KMeans Clustering
    
    takes female and male word's embeddings
    ground truth labels:
    0 = male,
    1 = female"""
    # print(males.shape, females.shape)
    array_m_f = np.concatenate((males,females))
    #print(array_m_f)
    
    #need: k (=1000) most biased female and male word's embedding (cosine similarity embedding & gender direction),
    #1. assign ground truth gender labels: 0 = male, 1 = female
    #2. run KMeans on embeddings
    kmeans = KMeans(n_clusters=2, random_state=0).fit(array_m_f)
    split = males.shape[0]
    
    #for i in kmeans.labels_:
    #    plt.scatter(np.concatenate((males, females))[kmeans.labels_ == i, 0], np.concatenate((males, females))[kmeans.labels_ == i, 1])
    #plt.show()
    
    # print(kmeans.labels_)
    # print(split)
    correct = 0
    #print(kmeans.labels_)
    #3. compute alignment score: cluster assignment vs ground truth gender label
    for i in range(array_m_f.shape[0]):
        if i < split and kmeans.labels_[i] == 0:
            correct+= 1
        elif i >= split and kmeans.labels_[i] == 1:
            correct += 1
    
    #4. alignment score = max(a, 1-a)
    alignment = 1/(2*array_m_f.shape[0]) * correct
    alignment = np.maximum(alignment, 1-alignment)
    return alignment 

Additional inputs: words to neutralize $N\subseteq W$, family of equality sets $\mathcal{E} = \{E_1, E_2, ..., E_m\}$ where each $E_i \subseteq W$. For each word $w \in N$, let $\vec{w}$ be re-embedded to $\vec{w}:=(\vec{w}-\vec{w}_B/||\vec{w}-\vec{w}_B||$. For each set $E\in \mathcal{E}$, let $\mu:=\sum_{w\in E}w/|E|$ and $v:=\mu-\mu_B$. For each $w \in E$, $\vec{w}:=v+\sqrt{1-||v||^2}\frac{\vec{w}_B-\mu_B}{||\vec{w}_B-\mu_B||}$. Finally, output the subspace $B$ and the new embedding $\{\vec{w}\in\mathbb{R}^d\}_{w\in W}$.

In [184]:
def hard_debias (word_emb, equality_sets=equalize_pairs, B=gender_subspace):
    """performs hard debias on a word embedding to neutralize it,
    
    takes 
    word_emb: word embedding of the word to be neutralized,
    equalize_pairs: equality pairs, each neutral word should be equidistant to all words in each equality set
    B: the bias subspace
    
    returns
    B: the bias subspace
    new_word_emb: the new embedding for word_emb
    """
    
    # if word_emb is a single embedding:
        # w_orth(word_emb)
        
    # if word_emb is the embeddings of all words to be neutralized:
    
    new_word_emb = np.zeros((word_emb.shape))
    for i, embedding in enumerate(word_emb):
        new_word_emb[i] = w_orth(embedding, B)
        
    for equal_set in equality_sets:
        if equal_set[0] in w2id and equal_set[1] in w2id:
            mean = (embed(equal_set[0]) + embed(equal_set[1])) / 2
            mean_biased = mean - w_orth(mean, B)
            v = mean - mean_biased # what is the biased mean?
            for word in equal_set:
                print(word)
                word_biased = embed(word) - w_orth(embed(word), B)
                new_embed = v + np.sqrt(1 - (np.linalg.norm(v)) ** 2) * ((word_biased - mean_biased) / unit_vec(word_biased - mean_biased))
    
    return new_word_emb#, B im paper steht, dass auch B returned werden soll, aber das macht hier keinen Sinn

In [185]:
def w_orth (word_emb, direction):
    """removes direction from word embedding by calculating the orthogonal word vector
    
    w_orth = w - (projection of w onto direction)
    w_orth = w - (direction * (w dot direction))
    
    takes 
    word_emb: word to remove the direction from
    direction: the direction to remove
    
    returns:
    embedding orthogonal to direction   
    """
    
    #new_word_emb_two = (word_emb) - ((word_emb @ np.transpose(unit_vec(direction))))
    
    # leads to evaluations of 0.5 and 1
    #new_word_emb = unit_vec(word_emb) - (direction *  unit_vec(word_emb).dot(direction))
    #print((np.dot(word_emb,direction)).shape)
    
    # formula from Bolukbasi et al. (2016)
    new_word = word_emb - ((word_emb.dot(direction)) * direction)
    

    # print(new_word_emb_two, new_word_emb, new)
    #print(new.shape)
    
    return unit_vec(new_word)

In [186]:
result_equal = double_hard_debias(embedding, index_m, index_f, w2id)

eval: [0.757, 0.768, 0.7404999999999999, 0.7090000000000001, 0.7515000000000001, 0.745, 0.751, 0.7515000000000001, 0.7515000000000001, 0.748, 0.748, 0.7515000000000001, 0.7484999999999999, 0.7525, 0.753, 0.747, 0.748, 0.752, 0.7505, 0.7484999999999999]
3
monastery
convent
spokesman
spokeswoman
dad
mom
men
women
councilman
councilwoman
grandpa
grandma
grandsons
granddaughters
testosterone
estrogen
uncle
aunt
husbands
wives
father
mother
grandpa
grandma
he
she
boy
girl
boys
girls
brother
sister
brothers
sisters
businessman
businesswoman
chairman
chairwoman
colt
filly
congressman
congresswoman
dad
mom
dads
moms
dudes
gals
father
mother
fatherhood
motherhood
fathers
mothers
fella
granny
fraternity
sorority
gelding
mare
gentleman
lady
gentlemen
ladies
grandfather
grandmother
grandson
granddaughter
he
she
himself
herself
his
her
king
queen
kings
queens
male
female
males
females
man
woman
men
women
nephew
niece
prince
princess
schoolboy
schoolgirl
son
daughter
sons
daughters


In [187]:
print(result_equal)

[[ 0.10805317 -0.08766379  0.01061857 ... -0.04900763  0.07792142
  -0.01665246]
 [ 0.07214007 -0.01246045  0.04986387 ... -0.02734615  0.06203784
  -0.06135967]
 [ 0.05096261 -0.02703599  0.07235463 ... -0.03247229  0.02815472
  -0.02323581]
 ...
 [-0.03249714 -0.03440436 -0.07950904 ... -0.00769115 -0.01051873
  -0.0550563 ]
 [ 0.0695056   0.04708673 -0.05388755 ...  0.06078932 -0.00027821
   0.05233467]
 [-0.16114553  0.06672763 -0.06129586 ...  0.06207156 -0.02941251
   0.0016239 ]]


In [188]:
print(result_equal.shape)

(314952, 300)


### Word Embedding Association Test

In [29]:
class weat(object):

    def __init__(self, concept1,concept2,stereotype1,stereotype2, iterations, embedding, w2id):
        self.concept1 = concept1
        self.concept2 = concept2
        self.stereotype1 = stereotype1
        self.stereotype2 = stereotype2
        self.iterations = iterations
        self.embedding = embedding
        self.w2id = w2id

    def getPValueAndEffect(self):
        pvalue = 0
        effect_size = 0
        sd = 0
        testStatistic = self.getTestStatistic(self.concept1,self.concept2,self.stereotype1,self.stereotype2, self.embedding, self.w2id)
        nullDist = self.nullDistribution(self.concept1, self.concept2, self.stereotype1, self.stereotype2, self.iterations, self.embedding, self.w2id)
        entireDistribution = self.getEntireDistribution(self.concept1, self.concept2, self.stereotype1, self.stereotype2, self.iterations, self.embedding, self.w2id)

        pvalue = 1-self.calculateCumulativeProbability(nullDist, testStatistic)
        effect_size = self.effectSize(entireDistribution, testStatistic)
        sd = stat.stdev(nullDist)
        return pvalue, effect_size, sd

    def nullDistribution(self, concept1, concept2, stereotype1, stereotype2, iterations, embedding, w2id):

        # permute concepts and for each permutation calculate getTestStatistic and save it in your distribution
        bothConcepts = concept1 + concept2
        print("Generating null distribution...")

        stereotype1NullMatrix = []
        stereotype2NullMatrix = []

        for attribute in stereotype1:
            similarity_list = []
            stereotype1Embedding = embedding[w2id[attribute]]

            for word in bothConcepts:
                nullEmbedding = embedding[w2id[word]]
                similarity = self.cosineSimilarity(nullEmbedding, stereotype1Embedding)
                similarity_list.append(similarity)
            stereotype1NullMatrix.append(similarity_list)


        for attribute in stereotype2:
            similarity_list = []
            stereotype2Embedding = embedding[w2id[attribute]]

            for word in bothConcepts:
                nullEmbedding = embedding[w2id[word]]
                similarity = self.cosineSimilarity(nullEmbedding, stereotype2Embedding)
                similarity_list.append(similarity)
            stereotype2NullMatrix.append(similarity_list)

        #Assuming both concepts have the same length
        setSize = int(len(bothConcepts)/2)
        print("Number of permutations ", iterations)
        toShuffle = list(range(0, len(bothConcepts)))
        distribution = []

        for iter in range(iterations):
            random.shuffle(toShuffle)
        	#calculate mean for each null shuffle
            meanSimilaritycon1str1 = 0
            meanSimilaritycon1str2 = 0
            meanSimilaritycon2str1 = 0
            meanSimilaritycon2str2 = 0

            for i in range(len(stereotype1)):
                for j in range(setSize):
                    meanSimilaritycon1str1 = meanSimilaritycon1str1 + stereotype1NullMatrix[i][toShuffle[j]]

            for i in range(len(stereotype2)):
                for j in range(setSize):
                    meanSimilaritycon1str2 = meanSimilaritycon1str2 + stereotype2NullMatrix[i][toShuffle[j]]

            for i in range(len(stereotype1)):
                for j in range(setSize):
                    meanSimilaritycon2str1 = meanSimilaritycon2str1 + stereotype1NullMatrix[i][toShuffle[j+setSize]]

            for i in range(len(stereotype2)):
                for j in range(setSize):
                    meanSimilaritycon2str2 = meanSimilaritycon2str2 + stereotype2NullMatrix[i][toShuffle[j+setSize]]

            meanSimilaritycon1str1 = meanSimilaritycon1str1/(len(stereotype1)*setSize)
            meanSimilaritycon1str2 = meanSimilaritycon1str2/(len(stereotype2)*setSize)
            meanSimilaritycon2str1 = meanSimilaritycon2str1/(len(stereotype1)*setSize)
            meanSimilaritycon2str2 = meanSimilaritycon2str2/(len(stereotype2)*setSize)

            #come back here later
            distribution.append((meanSimilaritycon1str1 - meanSimilaritycon1str2) - meanSimilaritycon2str1 + meanSimilaritycon2str2)

        return distribution

    def calculateCumulativeProbability(self,nullDistribution, testStatistic):
        cumulative = -100
        nullDistribution.sort()

        
        d = norm(loc = stat.mean(nullDistribution), scale = stat.stdev(nullDistribution))
        cumulative = d.cdf(testStatistic)

        return cumulative

    def effectSize(self,array, mean):
        effect = mean/stat.stdev(array)
        return effect

    def getTestStatistic(self, concept1, concept2, stereotype1, stereotype2, embedding, w2id):

        differenceOfMeans =0
        differenceOfMeansConcept1 =0
        differenceOfMeansConcept2 =0

        #concept 1 computations
        for word in concept1:
            concept1_embedding = embedding[w2id[word]]

            meanConcept1Stereotype1=0
            for attribute in stereotype1:
                stereotype1_embedding = embedding[w2id[attribute]]
                similarity = self.cosineSimilarity(concept1_embedding, stereotype1_embedding)
                meanConcept1Stereotype1 = meanConcept1Stereotype1 + similarity

            meanConcept1Stereotype1 = meanConcept1Stereotype1/len(stereotype1)


            meanConcept1Stereotype2=0
            for attribute in stereotype2:
                stereotype2_embedding = embedding[w2id[attribute]]
                similarity = self.cosineSimilarity(concept1_embedding, stereotype2_embedding)
                meanConcept1Stereotype2 = meanConcept1Stereotype2 + similarity

            meanConcept1Stereotype2 = meanConcept1Stereotype2/len(stereotype2)

            differenceOfMeansConcept1 = differenceOfMeansConcept1+ meanConcept1Stereotype1 - meanConcept1Stereotype2

        #effect size computations mean S(x,A,B)
        differenceOfMeansConcept1 = differenceOfMeansConcept1/len(concept1)

        #concept 2 computations
        for word in concept2:
            concept2_embedding = embedding[w2id[word]]

            meanConcept2Stereotype1=0
            for attribute in stereotype1:
                stereotype1_embedding = embedding[w2id[attribute]]
                similarity = self.cosineSimilarity(concept2_embedding, stereotype1_embedding)
                meanConcept2Stereotype1 = meanConcept2Stereotype1 + similarity

            meanConcept2Stereotype1 = meanConcept2Stereotype1/len(stereotype1)

            meanConcept2Stereotype2=0
            for attribute in stereotype2:
                stereotype2_embedding = embedding[w2id[attribute]]
                similarity = self.cosineSimilarity(concept2_embedding, stereotype2_embedding)
                meanConcept2Stereotype2 = meanConcept2Stereotype2 + similarity

            meanConcept2Stereotype2 = meanConcept2Stereotype2/len(stereotype2)

            differenceOfMeansConcept2 = differenceOfMeansConcept2+ meanConcept2Stereotype1 - meanConcept2Stereotype2

        #effect size computations mean S(x,A,B)
        differenceOfMeansConcept2 = differenceOfMeansConcept2/len(concept2)
        differenceOfMeans = differenceOfMeansConcept1 - differenceOfMeansConcept2

        #used for effect size computations before dividing by standard deviation
        print("The difference of means is ", differenceOfMeans)
        return differenceOfMeans

    def getEntireDistribution(self, concept1, concept2, stereotype1, stereotype2, iterations, embedding, w2id):

        bothConcepts = concept1 + concept2
        distribution = []
        print("Getting the entire distribution")

        for word in bothConcepts:
            conceptEmbedding = embedding[w2id[word]]
            similarityToStereotype1 = 0
            similarityToStereotype2 = 0

            for attribute in stereotype1:
                stereotype1Embedding = embedding[w2id[attribute]]
                similarityToStereotype1 = similarityToStereotype1 + self.cosineSimilarity(conceptEmbedding, stereotype1Embedding)
            similarityToStereotype1 = similarityToStereotype1/len(stereotype1)

            for attribute in stereotype2:
                stereotype2Embedding = embedding[w2id[attribute]]
                similarityToStereotype2 = similarityToStereotype2 + self.cosineSimilarity(conceptEmbedding, stereotype2Embedding)
            similarityToStereotype2 = similarityToStereotype2/len(stereotype2)

            distribution.append(similarityToStereotype1 - similarityToStereotype2)

        return distribution

    def cosineSimilarity(self,a, b):
        a = [a]
        b = [b]
        r = sk_m.pairwise.cosine_similarity(a,b)
        return r[0][0]

In [25]:
# Career and family
# Change from Bill to Tom as in paper to avoid ambiguity
male_names = ["john", "paul", "mike", "kevin", "steve", "greg", "jeff", "tom"]
female_names = ["amy", "joan", "lisa", "sarah", "diana", "kate", "ann", "donna"]
career_attributes = ["executive", "management", "professional", "corporation", "salary", "office", "business", "career"]
family_attributes = ["home", "parents", "children", "family", "cousins", "marriage", "wedding", "relatives"]

In [26]:
# Math and arts
math_words = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
arts_words1 = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
male_attributes1 = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female_attributes1 = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]

In [27]:
# Science and arts
science_words = ["science", "technology", "pyhsics", "chemistry", "einstein", "nasa", "experiment", "astronomy"]
arts_words2 = ["poetry", "art", "shakespeare", "dance", "literature", "novel", "symphony", "drama"]
male_attributes2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_attributes2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "her"]

In [30]:
iterations = 100000
wea_test = weat(male_names, female_names, career_attributes, family_attributes, iterations, embedding, w2id)
pvalue, effect_size, sd = wea_test.getPValueAndEffect()
print("p-value: ", pvalue)
print("effect size: ", effect_size)
print("standard deviation: ", sd)

The difference of means is  0.14427444139552204
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
p-value:  0.00015898145565074184
effect size:  1.805996189486473
standard deviation:  0.04008756771386324


KeyError: 'brother'